In [55]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords  
from nltk.stem.porter import PorterStemmer

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [16]:
tf.__version__

'2.4.1'

In [8]:
df =  pd.read_csv("/kaggle/input/fake-news/train.csv")

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df = df.dropna()

In [36]:
### Vocabulary size
voc_size=5000

In [13]:
x = df.drop('label', axis= 1)
y = df.label
x.shape, y.shape

((18285, 4), (18285,))

# One Hot representation

In [26]:
message = x.copy()

In [27]:
message.title[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [28]:
message.reset_index ( inplace =True)

In [34]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message.title[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [37]:
one_hot_rep = [one_hot(words , voc_size) for words in corpus]

In [39]:
sent_len = 20
embedded_docs = pad_sequences(one_hot_rep, padding='pre',maxlen = sent_len)

In [40]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  985,
       3432, 3998,  775, 3633, 2767,  739, 1485, 1483, 1905], dtype=int32)

In [42]:
embedding_vec_feature = 40
model =  Sequential()

model.add(Embedding(voc_size, embedding_vec_feature, input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss= 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [44]:
len(embedded_docs), y.shape

(18285, (18285,))

In [46]:
x_final = np.array(embedded_docs)
y_final = np.array(y)

x_final.shape, y_final.shape

((18285, 20), (18285,))

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size = 0.25, random_state = 200)

## Model Training



In [49]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 10, batch_size= 64)

Epoch 1/10
215/215 [==============================] - 9s 32ms/step - loss: 0.4658 - accuracy: 0.7575 - val_loss: 0.1884 - val_accuracy: 0.9224
Epoch 2/10
215/215 [==============================] - 6s 29ms/step - loss: 0.1438 - accuracy: 0.9424 - val_loss: 0.1891 - val_accuracy: 0.9256
Epoch 3/10
215/215 [==============================] - 6s 30ms/step - loss: 0.0962 - accuracy: 0.9652 - val_loss: 0.2124 - val_accuracy: 0.9254
Epoch 4/10
215/215 [==============================] - 6s 30ms/step - loss: 0.0687 - accuracy: 0.9769 - val_loss: 0.2531 - val_accuracy: 0.9245
Epoch 5/10
215/215 [==============================] - 6s 29ms/step - loss: 0.0417 - accuracy: 0.9867 - val_loss: 0.2671 - val_accuracy: 0.9009
Epoch 6/10
215/215 [==============================] - 6s 30ms/step - loss: 0.0394 - accuracy: 0.9891 - val_loss: 0.3379 - val_accuracy: 0.9136
Epoch 7/10
215/215 [==============================] - 6s 29ms/step - loss: 0.0217 - accuracy: 0.9930 - val_loss: 0.3552 - val_accuracy: 0.9156

## adding Dropout

In [61]:
embedding_vec_fet = 40
model = Sequential()

model.add ( Embedding(voc_size, embedding_vec_fet , input_length = sent_len))
model.add ( Dropout (0.3))
model.add ( LSTM (100))
model.add ( Dropout (0.3))
model.add ( Dense (1, activation = 'sigmoid'))

model.compile ( loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])

## performance metrics and accuracy

In [62]:
y_pred = model.predict_classes(x_test)


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [58]:
confusion_matrix(y_test , y_pred)

array([[1610,  986],
       [1121,  855]])

In [60]:
accuracy_score(y_test,  y_pred)

0.5391513560804899